### In this script, first we will clean the INSEE-format data. After that, we combine them with history voting data and export the final full data for analysis.
### INSEE datasets have identical format that's why we will use 1 process to clean them all. Next, the feature engineering is different for each data so we need to handle them seperately.

### ----------------- DATA CLEANING -----------------

In [1]:
import pandas as pd
import numpy as np

In [3]:
salary_data = pd.read_excel("salaire.xls", skiprows=5)
immigration_data = pd.read_excel("immigration.xls", skiprows=10)
population_data = pd.read_excel("population.xls", skiprows=5)
education_data = pd.read_excel("education.xls", skiprows=5)
logement_data = pd.read_excel("logement.xls", skiprows=5)

### First, as usual, we convert unicode to string.

In [4]:
import unicodedata

def unicode_to_string(word):
    if pd.isnull(word):
        return 
    else:
        return unicodedata.normalize('NFKD', word).encode('ascii','ignore')
    
def data_to_string(data):
    new_data = data.copy()
    
    col_title_unicode = new_data.columns.values.tolist()
    col_title = map(unicode_to_string,col_title_unicode)
    new_data.columns = col_title
    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == unicode: #check the first not-NaN value
            new_data[col] = map(unicode_to_string,new_data[col])
            
    return new_data

In [5]:
salary_data = data_to_string(salary_data)
immigration_data = data_to_string(immigration_data)
population_data = data_to_string(population_data)
education_data = data_to_string(education_data)
logement_data = data_to_string(logement_data)

### Now we will rename the column headers so that we have at least 1 common column for all the data frames. 

In [6]:
def rename_column(data): 
    new_data = data.copy()
    col_title = new_data.columns.tolist()
    for x in xrange(len(col_title)):
        if col_title[x] == 'CODGEO':
            col_title[x] = 'Code Insee' # this will be the pivot column for merging
        if col_title[x] == 'LIBGEO':
            col_title[x] = 'Libelle de la commune'
    new_data.columns = col_title
    return new_data

In [7]:
salary_data = rename_column(salary_data)
immigration_data = rename_column(immigration_data)
population_data = rename_column(population_data)
education_data = rename_column(education_data)
logement_data = rename_column(logement_data)

### We still have problem with departemental code of Corse because they are not in numerical form (2A... or 2B...).

In [8]:
def to_digit(string):
    new_string = list(string)[:]
    for x in xrange(len(new_string)): 
        if not new_string[x].isdigit():
            new_string[x] = '0'
    return int(''.join(new_string))

In [9]:
def replace_insee_code(data): #replace 2AXXX or 2BXXX by 20XXX 
        new_data = data.copy()
        insee_list = new_data['Code Insee'].tolist() 
        new_insee_list = [int(code) if code.isdigit() else to_digit(code) for code in insee_list]
        new_data['Code Insee'] = new_insee_list
        return new_data

In [10]:
salary_data = replace_insee_code(salary_data)
immigration_data = replace_insee_code(immigration_data)
population_data = replace_insee_code(population_data)
education_data = replace_insee_code(education_data)
logement_data = replace_insee_code(logement_data)

### There is still another problem: all the numbers in our excel sheet are read in float format. We need to convert them to int for the immigration and population data because they contain quantity-format data.

In [11]:
def float_to_int(data):
    new_data = data.copy()    
    for col in new_data.columns:
        not_nan_index = [not ind for ind in new_data[col].isnull()]
        not_nan_value = new_data[col][not_nan_index]
        if type(not_nan_value.iloc[0]) == np.float64: #check the first not-NaN value
            new_data[col] = new_data[col].round()
    return new_data

In [12]:
immigration_data = float_to_int(immigration_data)
population_data = float_to_int(population_data)
education_data = float_to_int(education_data)
logement_data = float_to_int(logement_data )

### ----------------- FEATURE ENGINEERING -----------------

### In immigration data, we will add a column which shows us the percentage of adult immigrants  in each town.

In [13]:
def add_total_immigrant_variable(data): 
    new_data = data.copy()
    variable_list = []
    sum_list = []
    for variable in data.columns.tolist(): 
        if 'IMMI' in variable and 'AGE400' not in variable: 
            sum_list.append(variable)
            if 'IMMI1' in variable:
                variable_list.append(variable)
    new_data['% immigrant'] = (new_data[variable_list].sum(axis=1)/ new_data[sum_list].sum(axis=1)) * 100
    col = new_data.columns.tolist()
    col = col[:2] + col[-1:] + col[2:-1]
    new_data = new_data[col]
    return new_data

In [14]:
immigration_data = add_total_immigrant_variable(immigration_data)

In [16]:
immigration_data.head()

,Code Insee,Libelle de la commune,% immigrant,AGE400_IMMI1_SEXE1,AGE400_IMMI1_SEXE2,AGE400_IMMI2_SEXE1,AGE400_IMMI2_SEXE2,AGE415_IMMI1_SEXE1,AGE415_IMMI1_SEXE2,AGE415_IMMI2_SEXE1,AGE415_IMMI2_SEXE2,AGE425_IMMI1_SEXE1,AGE425_IMMI1_SEXE2,AGE425_IMMI2_SEXE1,AGE425_IMMI2_SEXE2,AGE455_IMMI1_SEXE1,AGE455_IMMI1_SEXE2,AGE455_IMMI2_SEXE1,AGE455_IMMI2_SEXE2
0,1001,L'Abergement-Clemenciat,4.173623,1,3,92,83,1,0,34,32,7,9,154,137,5,3,101,116
1,1002,L'Abergement-de-Varey,2.105263,0,1,25,18,1,0,6,11,2,1,52,47,0,0,38,32
2,1004,Amberieu-en-Bugey,12.048946,57,53,1386,1542,73,61,802,872,442,403,2447,2386,194,176,1454,1886
3,1005,Amberieux-en-Dombes,4.528012,0,0,176,163,0,2,108,101,18,19,334,324,11,9,182,195
4,1006,Ambleon,8.000000,2,1,5,4,0,0,8,3,2,3,23,19,0,3,18,21


### In education data, we convert the data to percentage. 

In [17]:
def education_level_in_percentage(data):
    new_data = data.copy()
    variable_list = ['Code Insee', 'Libelle de la commune']
    for variable in new_data.columns.tolist(): 
        if '_NSCOL15P_' in variable:
            variable_list.append(variable)
    for chosen_variable in variable_list[2:]:
        new_data[chosen_variable] = (new_data[chosen_variable] / new_data['P12_NSCOL15P']) * 100
        #new_data[chosen_variable] = new_data['P12_NSCOL15P'] - new_data[variable_list].sum(axis=1)
    return new_data[variable_list]

In [18]:
education_data = education_level_in_percentage(education_data)

In [19]:
education_data.head()

,Code Insee,Libelle de la commune,P12_NSCOL15P_DIPL0,P12_NSCOL15P_CEP,P12_NSCOL15P_BEPC,P12_NSCOL15P_CAPBEP,P12_NSCOL15P_BAC,P12_NSCOL15P_BACP2,P12_NSCOL15P_SUP
0,1001,L'Abergement-Clemenciat,14.000000,13.454545,5.818182,25.272727,16.909091,15.272727,9.454545
1,1002,L'Abergement-de-Varey,8.888889,6.666667,3.333333,22.222222,18.333333,21.111111,18.888889
2,1004,Amberieu-en-Bugey,17.637173,9.362333,5.783490,24.824518,19.110232,13.109244,10.173010
3,1005,Amberieux-en-Dombes,16.016949,12.033898,6.186441,29.491525,16.016949,11.610169,8.644068
4,1006,Ambleon,15.555556,7.777778,8.888889,32.222222,22.222222,6.666667,7.777778


### In population data, we will transform the social status columns from quantity to percentage too.

In [20]:
def social_status_percentage(data): 
    new_data = data.copy()
    status_list = ['C12_POP15P_CS1', 'C12_POP15P_CS2','C12_POP15P_CS3','C12_POP15P_CS4','C12_POP15P_CS5',
                   'C12_POP15P_CS6','C12_POP15P_CS7','C12_POP15P_CS8']
    adult_population = new_data['C12_POP15P']
    for col in status_list:
        new_data['% '+ col] = (new_data[col] / adult_population)*100
    return new_data

In [21]:
population_data = social_status_percentage(population_data)

In [22]:
population_data.head()

,Code Insee,REG,DEP,Libelle de la commune,P12_POP,P12_POP0014,P12_POP1529,P12_POP3044,P12_POP4559,P12_POP6074,...,C12_POP55P_CS7,C12_POP55P_CS8,% C12_POP15P_CS1,% C12_POP15P_CS2,% C12_POP15P_CS3,% C12_POP15P_CS4,% C12_POP15P_CS5,% C12_POP15P_CS6,% C12_POP15P_CS7,% C12_POP15P_CS8
0,1001,82,01,L'Abergement-Clemenciat,777,178,97,165,151,128,...,163,8,2.588997,0.647249,12.944984,14.239482,18.770227,11.650485,27.022654,12.297735
1,1002,82,01,L'Abergement-de-Varey,235,44,35,60,38,36,...,64,0,0.000000,2.116402,10.582011,27.513228,4.232804,16.931217,33.862434,4.232804
2,1004,82,01,Amberieu-en-Bugey,14233,3038,2904,2722,2655,1737,...,2816,251,0.000000,2.818540,6.451324,15.595920,17.143880,17.197566,25.590551,15.211167
3,1005,82,01,Amberieux-en-Dombes,1642,338,301,348,359,203,...,321,33,0.000000,5.611068,8.147579,17.524981,18.139892,14.681015,25.288240,10.607225
4,1006,82,01,Ambleon,110,12,18,24,27,20,...,35,0,0.000000,0.000000,4.651163,18.604651,18.604651,9.302326,40.697674,9.302326


### In logement data, we compute the percentage of house proprietor

In [23]:
def proprietor_percentage(data):
    new_data = data.copy()
    new_data['P12_NPER_RP_PROP'] = ((new_data['P12_NPER_RP_PROP'])/new_data['P12_NPER_RP'])*100
    return new_data

In [24]:
logement_data = proprietor_percentage(logement_data)

### -------------------------------------------------------------------- DATA MERGING -----------------------------------------------------------------

### Now once we have all the ingredients needed, we can start merging them

In [88]:
def drop_col(data):
    new_data = data.copy()
    title = data.columns.tolist()
    dropping_index = []
    voting_col = [col for col in new_data.columns.tolist() if 'vote' in col]
    keeping_variable = ['Code Insee'] + voting_col +['% immigrant','% C12_POP15P_CS1','% C12_POP15P_CS2','% C12_POP15P_CS3',
                        '% C12_POP15P_CS4','% C12_POP15P_CS5', '% C12_POP15P_CS6','% C12_POP15P_CS7', '% C12_POP15P_CS8',
                        'SNHM12','P12_NSCOL15P_DIPL0','P12_NSCOL15P_CEP','P12_NSCOL15P_BEPC','P12_NSCOL15P_CAPBEP','P12_NSCOL15P_BAC',
                        'P12_NSCOL15P_BACP2','P12_NSCOL15P_SUP','P12_NPER_RP_PROP']
    for index, variable in enumerate(title): 
        if variable not in keeping_variable:
            dropping_index.append(index)
    new_data = new_data.drop(new_data.columns[dropping_index], axis = 1)
    return new_data

In [103]:
def merge_data(insee_data_list,voting_history_file_list, actual_voting_file_list, export_name):
    insee_data = insee_data_list[0]
    for x in xrange(1,len(insee_data_list)):
        insee_data = pd.merge(insee_data,insee_data_list[x], on = ['Code Insee','Libelle de la commune'],
                              how='outer') # outer means that we keep a row if either one of the 2 dataset is not null
    for voting_file in actual_voting_file_list: 
        if 'right' in voting_file: 
            full_data_right = pd.read_excel(voting_file)
        elif 'left' in voting_file: 
            full_data_left = pd.read_excel(voting_file)
        else: 
            full_data_er = pd.read_excel(voting_file)
    
    for voting_history_file in voting_history_file_list:
        if 'right' in voting_history_file: 
            voting_history_right = pd.read_excel(voting_history_file)
            full_data_right = pd.merge(full_data_right, voting_history_right, on = 'Code Insee', how = 'left')
        elif 'left' in voting_history_file: 
            voting_history_left = pd.read_excel(voting_history_file)
            full_data_left = pd.merge(full_data_left, voting_history_left, on = 'Code Insee', how = 'left')
        else:
            voting_history_er = pd.read_excel(voting_history_file)
            full_data_er = pd.merge(full_data_er, voting_history_er, on = 'Code Insee', how = 'left')
    
    full_data_right = pd.merge(full_data_right, insee_data, on = 'Code Insee', how = 'left')
    full_data_left = pd.merge(full_data_left, insee_data, on = 'Code Insee', how = 'left')
    full_data_er = pd.merge(full_data_er, insee_data, on = 'Code Insee', how = 'left')
    
    full_data_right = drop_col(full_data_right)
    full_data_left = drop_col(full_data_left)
    full_data_er = drop_col(full_data_er)
    
    return full_data_right, full_data_left, full_data_er

In [93]:
insee_data_list = [immigration_data, salary_data, population_data, education_data, logement_data]
voting_history_file_list = ['regionale2004_right.xlsx','regionale2004_left.xlsx','regionale2004_er.xlsx',
                            'regionale2010_right.xlsx','regionale2010_left.xlsx', 'regionale2010_er.xlsx']
actual_voting_file_list = ['regionale2015_left.xlsx','regionale2015_right.xlsx','regionale2015_er.xlsx']

In [94]:
right,left,er = merge_data(insee_data_list,voting_history_file_list,actual_voting_file_list, 'coucou')

In [105]:
columns_list = ['Immigrant','Salaire','Agriculteurs','Artisans','Cadres','Prof_Intermediaire','Employes',
                'Ouvrier', 'Retraite', 'Autre','Sans diplome','CEP','BEPC','CAP-BEP','BAC','Enseignement_sup_court',
                'Enseignement_sup_long', 'Proprietaire'] 
                # this list must be in the order of the data's column list 
right.columns = right.columns.tolist()[:4] + columns_list
left.columns = left.columns.tolist()[:4] + columns_list
er.columns = er.columns.tolist()[:4] + columns_list

#### (We would love to put the above cell in a function, but it is quite tricky as we don't know the order of the columns. A revoir...)

In [108]:
writer = pd.ExcelWriter('analyze_data_right.xlsx')
right.to_excel(writer,'Sheet1')
writer.save()
    
writer = pd.ExcelWriter('analyze_data_left.xlsx')
left.to_excel(writer,'Sheet1')
writer.save()
    
writer = pd.ExcelWriter('analyze_data_er.xlsx')
er.to_excel(writer,'Sheet1')
writer.save()

In [84]:
import cProfile

cProfile.run("merge_data(insee_data_list, voting_history_file_list, actual_voting_file_list,'coucou')")

         944224880 function calls (918204609 primitive calls) in 586.311 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    1.861    1.861  586.428  586.428 <ipython-input-80-299487c78c53>:1(merge_data)
        1    0.000    0.000  586.428  586.428 <string>:1(<module>)
       18    0.000    0.000    0.000    0.000 <string>:109(__iter__)
       36    0.000    0.000    0.003    0.000 <string>:23(parse)
 2241/567    0.001    0.000    0.001    0.000 <string>:44(iter)
       36    0.000    0.000    0.004    0.000 <string>:60(parse)
       18    0.000    0.000    0.000    0.000 <string>:67(__init__)
 22588055    7.320    0.000   13.792    0.000 <string>:8(__new__)
  2215230    0.699    0.000    2.198    0.000 <string>:80(next)
       36    0.000    0.000    0.000    0.000 ElementTree.py:607(__init__)
       27    0.000    0.000    0.000    0.000 ElementTree.py:671(iter)
        9    0.000    0.000    0.000    0.000 Elem

KeyboardInterrupt: 